# [Go Gin 简明教程](https://geektutu.com/post/quick-go-gin.html)
## 路由（Router）
- 路由方法有：`GET`, `POST`, `PUT`, `PATCH`, `DELETE` 和 `OPTIONS`, 还有 `Any`
### 无参数

In [1]:
// 无参数
r.GET("/", func(c *gin.Context) {
    c.String(http.StatusOK, "Who are you ?")
})

ERROR: repl.go:2:1: undefined identifier: r

In [ ]:
$ curl http://localhost:9999/
Who are you ?

### 解析路径函数
- 有时候我们需要动态的路由，如 `/user/:name` ，通过调用不同的url来传入不同的name。
- `/user/:name/*role`, `*`代表可选。

In [ ]:
// 匹配 /user/geektutu
r.GET("/user/:name", func(c *gin.Context) {
    name := c.Param("name")
    c.String(http.StatusOK, "Hello %s", name)
})

In [ ]:
$ curl http://localhost:9999/user/geektutu
Hello geektutu

### 获取Query参数

In [ ]:
// 匹配users?name=xxx&role=xxx，role可选
r.GET("/users", func(c *gin.Context) {
    name := c.Query("name")
    role := c.DefaultQuery("role", "teacher")
    c.String(http.StatusOK, "%s is a %s", name, role)
})

In [ ]:
$ curl "http://localhost:9999/users?name=Tom&role=sutdent"
Tom is a student

### 获取POST参数

In [ ]:
// POST
r.POST("/form", func(c *gin.Context) {
    username := c.PostForm("username")
    password := c.DefaultPostForm("password", "000000")

    c.JSON(http.StatusOK, gin.H {
        "username": username,
        "password": password,
    })
})

In [ ]:
$ curl http://localhost:9999/form  -X POST -d 'username=geektutu&password=1234'
{"password":"1234","username":"geektutu"}

### Query 和 POST 混合参数

In [ ]:
// GET 和 POST 混合
r.POST("/posts", func(c *gin.Context) {
    id := c.Query("id")
    page := c.DefaultQuery("page", "0")
    username := c.PostForm("username")
    password := c.DefaultPostForm("username", "000000") // 可设置默认值

    c.JSON(http.StatusOK, gin.H{
        "id":       id,
        "page":     page,
        "username": username,
        "password": password,
    })
})

In [ ]:
$ curl "http://localhost:9999/posts?id=9876&page=7"  -X POST -d 'username=geektutu&password=1234'
{"id":"9876","page":"7","password":"1234","username":"geektutu"}

### Map 参数

In [ ]:
r.POST("/post", func(c *gin.Context) {
    ids := c.QueryMap("ids")
    names := c.PostFormMap("names")

    c.JSON(http.StatusOK, gin.H {
        "ids": ids,
        "names": names,
    })
})

In [ ]:
$ curl -g "http://localhost:9999/post?ids[Jack]=001&ids[Tom]=002" -X POST -d 'names[a]=Sam&names[b]=David'
{"ids":{"Jack":"001","Tom":"002"},"names":{"a":"Sam","b":"David"}}

### 重定向

In [ ]:
r.GET("/redirect", func(c *gin.Context) {
    c.Redirect(http.StatusMovedPermanently, "/index")
})

r.GET("/goindex", func(c *gin.Context) {
    c.Request.URL.Path = "/"
    r.HandleContext(c)
})

In [ ]:
$ curl -i http://localhost:9999/redirect
HTTP/1.1 301 Moved Permanently
Content-Type: text/html; charset=utf-8
Location: /
Date: Thu, 08 Aug 2019 17:22:14 GMT
Content-Length: 36

<a href="/">Moved Permanently</a>.

$ curl "http://localhost:9999/goindex"
Who are you?